In [80]:
import os
import pandas as pd

dataframes = pd.DataFrame()

base1 = pd.read_csv("D:\\CEDERJ\\2024.2\\tcc\\IVOptionPredictor\\projeto\\data\\iniciais\\strike_4a10_spot_3.5a12.2\\base_de_dados_1.csv")
results1 = pd.read_csv("D:\\CEDERJ\\2024.2\\tcc\\IVOptionPredictor\\projeto\\data\\iniciais\\strike_4a10_spot_3.5a12.2\\result_1.csv")
        
# df_result_filtrado = results1[(results1['Result'].notna()) & (results1['Result'] != -1)]
# indices_to_keep = df_result_filtrado.index
# df_inputs_filtrado = base1.loc[indices_to_keep]
# df_inputs_filtrado['Result'] = df_result_filtrado['Result']

# dataframes = pd.concat([dataframes, df_inputs_filtrado], axis = 0)

df_base1 = base1
df_base1['Result'] = results1['Result']
dataframes = pd.concat([dataframes, df_base1], axis = 0)

base2 = pd.read_csv("D:\\CEDERJ\\2024.2\\tcc\\IVOptionPredictor\\projeto\\data\\iniciais\\strike_10a15_spot_8.5a17.2\\base_de_dados_2.csv")
results2 = pd.read_csv("D:\\CEDERJ\\2024.2\\tcc\\IVOptionPredictor\\projeto\\data\\iniciais\\strike_10a15_spot_8.5a17.2\\result_2.csv")

df_base2 = base2
df_base2['Result'] = results2['Result']
dataframes = pd.concat([dataframes, df_base2], axis = 0)

base3 = pd.read_csv("D:\\CEDERJ\\2024.2\\tcc\\IVOptionPredictor\\projeto\\data\\iniciais\\strike_15a20_spot_12.5a22.5\\base_de_dados_3.csv")
results3 = pd.read_csv("D:\\CEDERJ\\2024.2\\tcc\\IVOptionPredictor\\projeto\\data\\iniciais\\strike_15a20_spot_12.5a22.5\\result_3.csv")

df_base3 = base3
df_base3['Result'] = results3['Result']
dataframes = pd.concat([dataframes, df_base3], axis = 0)

dataframes['CallPut'] = dataframes['CallPut'].replace({'call': 0, 'put': 1})

dataframes_testes = dataframes.copy()


<h1>Estudo dados</h1>

In [ ]:
import matplotlib as plt

df_analise_dados_maturity_02 = dataframes[(dataframes['Maturity'] == 0.2) & (dataframes['TaxaDesconto'] == 0.08) & (dataframes['Spot'] == 15)]

df_analise_dados_maturity_02['result'] = df_analise_dados_maturity_02['result'].replace(-1, 0)

plt.figure(figsize=(10, 6))
plt.scatter(df_analise_dados_maturity_02['Strike'], df_analise_dados_maturity_02['Result'], alpha=0.5)
plt.xlabel('Strike')
plt.ylabel('Result')
plt.title('Scatter Plot: Result vs Strike')
plt.grid(True)
plt.show()

<h1>Filtrando dataframe para execução do grid search</h1>

In [38]:
print(dataframes.shape[0])
df_filtrado = dataframes[dataframes['TaxaDesconto'].isin([0.08, 0.09, 0.1])]
print(df_filtrado.shape[0])
df_filtrado = df_filtrado[df_filtrado['Maturity'].isin([0.2, 0.3, 0.4])]
print(df_filtrado.shape[0])
# df_filtrado = df_filtrado[df_filtrado['CallPut'].isin([0])]
# print(df_filtrado.shape[0])

df_filtrado = df_filtrado[(df_filtrado['Spot'] >= 9.5) & (df_filtrado['Spot'] <= 15.5)]
# print(df_filtrado.shape[0])

dataframes_filtrado = df_filtrado
print(dataframes.shape[0])

4227193
2536201
810060
356855


<h1>Execução do grid search</h1>

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Dados de entrada e variável alvo
X = dataframes[['Strike', 'Maturity', 'Spot', 'TaxaDesconto', 'Premio', 'CallPut']].values
y = dataframes['Result'].values

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir o pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalizar os dados
    ('model', LinearRegression())  # Placeholder para o modelo, será substituído
])

# Dicionário com modelos e seus hiperparâmetros
param_grid = [
    {
        'model': [RandomForestRegressor()],
        'model__n_estimators': [50, 100],
        'model__max_depth': [None, 10],
        'model__min_samples_split': [5, 10]
    },
    {
        'model': [SVR()],
        'model__kernel': ['rbf'],
        'model__C': [1.0, 10.0]
    }
]

# GridSearchCV para encontrar o melhor modelo e hiperparâmetros
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Ajuste do GridSearch com os dados de treino
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

print("Melhor modelo e hiperparâmetros:", grid_search.best_params_)

# Avaliação no conjunto de teste
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE no conjunto de teste: {mse}")
print(f"R² no conjunto de teste: {r2}")

# Exemplo de previsão com o melhor modelo
# new_data = np.array([[strike, maturity, spot, taxa_desconto, premio, callput]])
# predicted_vol = best_model.predict(new_data)
# print(f"Volatilidade implícita estimada: {predicted_vol}")


Melhor modelo e hiperparâmetros: {'model': RandomForestRegressor(), 'model__max_depth': None, 'model__min_samples_split': 5, 'model__n_estimators': 100}
MSE no conjunto de teste: 0.0002564788697873948
R² no conjunto de teste: 0.9990937358377502


<h1>Treinamento do modelo random forest</h1>

In [81]:
#### LIMPEZA DOS DADOS
dataframes_RF = dataframes.copy()

df_result_filtrado_RF = dataframes_RF[(dataframes_RF['Result'].notna()) & (dataframes_RF['Result'] != -1)]
# indices_to_keep = df_result_filtrado.index
# df_inputs_filtrado = base3.loc[indices_to_keep]
# df_inputs_filtrado['Result'] = df_result_filtrado['Result']

In [83]:
X = df_result_filtrado_RF[['Strike', 'Maturity', 'Spot', 'TaxaDesconto', 'Premio', 'CallPut']].values
y = df_result_filtrado_RF['Result'].values

X_train_RF, X_test_RF, y_train_RF, y_test_RF = train_test_split(X, y, test_size=0.3, random_state=42)

scaler_X_RF = StandardScaler()
scaler_y_RF = StandardScaler()
model_RF = RandomForestRegressor()

# Ajustar e transformar os dados de treino
X_train_scaled_RF = scaler_X_RF.fit_transform(X_train_RF)
y_train_scaled_RF = scaler_y_RF.fit_transform(y_train_RF.reshape(-1, 1))

X_test_scaled_RF = scaler_X_RF.fit_transform(X_test_RF)

In [85]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error, r2_score

rf_model = RandomForestRegressor(
    max_depth=None, 
    min_samples_split=5, 
    n_estimators=100,
    random_state=42
)

# Definir o KFold para a validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Usar cross_val_score para calcular o MSE e R² em cada dobra
mse_scorer_RF = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer_RF = make_scorer(r2_score)

mse_scores_RF = cross_val_score(rf_model, X_train_scaled_RF, y_train_scaled_RF, cv=kf, scoring=mse_scorer_RF)
r2_scores_RF = cross_val_score(rf_model, X_train_scaled_RF, y_train_scaled_RF, cv=kf, scoring=r2_scorer_RF)

# Ajustar o modelo aos dados de treino
rf_model.fit(X_train_scaled_RF, y_train_scaled_RF)

# Fazer previsões no conjunto de teste
y_pred_RF_scaled = rf_model.predict(X_test_scaled_RF)
y_pred_RF = scaler_y_RF.inverse_transform(y_pred_RF_scaled)

# Avaliar o modelo no conjunto de teste
mse_test_RF = mean_squared_error(y_test_RF, y_pred_RF)
r2_test_RF = r2_score(y_test_RF, y_pred_RF)

# Resultados da validação cruzada e do conjunto de teste
print(f"MSE médio (validação cruzada): {-np.mean(mse_scores_RF)}")
print(f"R² médio (validação cruzada): {np.mean(r2_scores_RF)}")
print(f"MSE no conjunto de teste: {mse_test_RF}")
print(f"R² no conjunto de teste: {r2_test_RF}")


d:\CEDERJ\2024.2\tcc\IVOptionPredictor\myenv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
d:\CEDERJ\2024.2\tcc\IVOptionPredictor\myenv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
d:\CEDERJ\2024.2\tcc\IVOptionPredictor\myenv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
d:\CEDERJ\2024.2\tcc\IVOptionPredictor\myenv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed

KeyboardInterrupt: 

<h1>Estudo do modelo Random Rorest Regressor</h1>

In [79]:
import matplotlib.pyplot as plt
import pandas as pd
df_RF_estudo = dataframes.copy()

df_estudo_vencimento = df_RF_estudo[(df_RF_estudo['Maturity'] == 0.2) & (df_RF_estudo['TaxaDesconto'] == 0.08) & (df_RF_estudo['Spot'] == 15)]

dados_x_RF = df_estudo_vencimento[['Strike', 'Maturity', 'Spot', 'TaxaDesconto', 'Premio', 'CallPut']].values

dados_x_RF_scaled = scaler_X_RF.fit_transform(dados_x_RF)

y_pred_estudo_RF = rf_model.predict(dados_x_RF_scaled)

y_estudo_reescalado = scaler_y_RF.inverse_transform(y_pred_RF)
scaler_X_RF.inverse_transform(y_estudo_reescalado.reshape(-1, 1))

plt.figure(figsize=(10, 6))
plt.scatter(df_estudo_vencimento['Strike'], y_pred_estudo_RF, alpha=0.5)
plt.xlabel('Strike')
plt.ylabel('Result')
plt.title('Scatter Plot: Result vs Strike')
plt.grid(True)
plt.show()


NameError: name 'scaler_X_RF' is not defined

<h1>grid search do modelo sgboost</h1>

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Dados de entrada e variável alvo
X = dataframes[['Strike', 'Maturity', 'Spot', 'TaxaDesconto', 'Premio', 'CallPut']].values
y = dataframes['Result'].values

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir o pipeline
pipeline2 = Pipeline([
    ('scaler', StandardScaler()),  # Normalizar os dados
    ('model', XGBRegressor())  # Substituir o modelo pelo XGBRegressor
])

# Dicionário com hiperparâmetros para o XGBoost
param_grid2 = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [None, 10, 5],
    'model__learning_rate': [0.01, 0.1],
    'model__subsample': [0.8, 1.0]
}

# GridSearchCV para encontrar o melhor modelo e hiperparâmetros
grid_search2 = GridSearchCV(pipeline2, param_grid2, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Ajuste do GridSearch com os dados de treino
grid_search2.fit(X_train, y_train)

# Melhor modelo encontrado
best_model2 = grid_search2.best_estimator_

print("Melhor modelo e hiperparâmetros:", grid_search2.best_params_)

# Avaliação no conjunto de teste
y_pred = best_model2.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE no conjunto de teste: {mse}")
print(f"R² no conjunto de teste: {r2}")

# Exemplo de previsão com o melhor modelo
# new_data = np.array([[strike, maturity, spot, taxa_desconto, premio, callput]])
# predicted_vol = best_model.predict(new_data)
# print(f"Volatilidade implícita estimada: {predicted_vol}")


Melhor modelo e hiperparâmetros: {'model__learning_rate': 0.1, 'model__max_depth': 10, 'model__n_estimators': 100, 'model__subsample': 0.8}
MSE no conjunto de teste: 0.0005828655999931901
R² no conjunto de teste: 0.9979921296692095


<h1>Treinamento do sgboost</h1>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Dados de entrada e variável alvo
X = dataframes[['Strike', 'Maturity', 'Spot', 'TaxaDesconto', 'Premio', 'CallPut']].values
y = dataframes['Result'].values

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir o pipeline com StandardScaler e XGBRegressor
pipeline_sgboost = Pipeline([
    ('scaler', StandardScaler()),  # Normalizar os dados
    ('model', XGBRegressor(
        learning_rate=0.1,
        max_depth=10,
        n_estimators=100,
        subsample=0.8
    ))  # Utilizando os hiperparâmetros fornecidos
])

# Definir o KFold para a validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Usar cross_val_score para calcular o MSE e R² em cada dobra
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score)

mse_scores = cross_val_score(pipeline_sgboost, X_train, y_train, cv=kf, scoring=mse_scorer)
r2_scores = cross_val_score(pipeline_sgboost, X_train, y_train, cv=kf, scoring=r2_scorer)

# Treinar o modelo com os dados de treino
pipeline_sgboost.fit(X_train, y_train)

# Avaliação do modelo no conjunto de teste
y_pred = pipeline_sgboost.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE no conjunto de teste: {mse}")
print(f"R² no conjunto de teste: {r2}")

# Exemplo de previsão com o melhor modelo
# new_data = np.array([[strike, maturity, spot, taxa_desconto, premio, callput]])
# predicted_vol = pipeline.predict(new_data)
# print(f"Volatilidade implícita estimada: {predicted_vol}")


<h1>Estudo do modelo sgboost</h1>